In [1]:
# %% ライブラリインポート
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [2]:
# %% データ読み込み
file_path = "/workspace/data/raw/StatsAllGroup.csv"

try:
    df = pd.read_csv(file_path, encoding="cp932")
    print(f"✅ データ読み込み成功: {df.shape[0]} 行, {df.shape[1]} 列")
except Exception as e:
    raise RuntimeError(f"❌ 読み込みエラー: {e}")


✅ データ読み込み成功: 68172 行, 71 列


In [3]:
# %% 欠損値処理（完全欠損行＋必須列のみチェック）
print("\n--- 欠損値処理 ---")

# 完全欠損行の削除（安全）
df = df.dropna(how="all")

# 分析に必須な列だけを指定
required_cols = [
    "date_",
    "athlete_id",
    "athlete_name",
    "start_time",
    "end_time",
    "total_distance",
]

rows_before = len(df)
df = df.dropna(subset=required_cols)
rows_after = len(df)

print(f"削除前の行数: {rows_before}")
print(f"削除後の行数: {rows_after}")
print(f"削除された行数: {rows_before - rows_after}")



--- 欠損値処理 ---
削除前の行数: 68172
削除後の行数: 68171
削除された行数: 1


In [4]:
unique_before = df['athlete_name'].nunique()
print(f"統一前のユニーク選手数: {unique_before}")

統一前のユニーク選手数: 75


In [5]:
# %% 選手名の統一・整形
print("\n--- 選手名の統一・整形 ---")

name_mapping = {
    "Ryu NAGAI": "Ryo NAGAI",
    "Asai YADA": "Asahi YADA",
    "Kakeru Sakamoto": "Kakeru SAKAMOTO",
    "Kozi SUGIYAMA": "Koji SUGIYAMA",
    "Sunjin KO": "Seungjin KO",
    "1 練習生": "Trainee 1",
    "2 練習生": "Trainee 2",
    "A Trainee": "Trainee A",
    "B Trainee": "Trainee B",
    "C Trainee": "Trainee C",
    "D Trainee": "Trainee D",
    "予備 YOBI": "Yobi 1",
    "予備2 Yobi2": "Yobi 2",
}

df["athlete_name"] = (
    df["athlete_name"]
    .replace(name_mapping)
    .str.strip()
    .str.replace("\u3000", " ", regex=True)
)

def format_name(name):
    if "Trainee" in name or "Yobi" in name:
        return name
    parts = name.split()
    if len(parts) >= 2:
        first, last = parts[0].capitalize(), parts[1].upper()
        return f"{first} {last}"
    return name

df["athlete_name"] = df["athlete_name"].apply(format_name)
print("✅ 選手名整形完了")


--- 選手名の統一・整形 ---
✅ 選手名整形完了


In [6]:
unique_after = df['athlete_name'].nunique()
print(f"統一後のユニーク選手数: {unique_after}")

統一後のユニーク選手数: 70


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68171 entries, 0 to 68171
Data columns (total 71 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   date_                                           68171 non-null  object 
 1   date_name                                       68171 non-null  object 
 2   start_time                                      68171 non-null  float64
 3   start_time_h                                    68171 non-null  object 
 4   end_time                                        68171 non-null  float64
 5   end_time_h                                      68171 non-null  object 
 6   activity_id                                     68171 non-null  object 
 7   activity_name                                   68171 non-null  object 
 8   period_id                                       68171 non-null  object 
 9   period_name                                 

In [8]:
# %% 型変換と日付処理
print("\n--- 型変換と日付整形 ---")

# 日付列: 文字列からdatetimeに変換
df["date_"] = pd.to_datetime(df["date_"], errors="coerce")

# date_nameが完全にdate_と一致する場合は削除
if "date_name" in df.columns:
    same_dates = (pd.to_datetime(df["date_name"], errors="coerce") == df["date_"]).all()
    if same_dates:
        df = df.drop(columns=["date_name"])
        print("🗑️ 'date_name' は 'date_' と重複していたため削除しました。")
    else:
        print("⚠️ 'date_name' に 'date_' と異なる値が含まれています。手動確認を推奨。")

# カテゴリ列
category_cols = ["activity_name", "period_name", "day_code"]
for col in category_cols:
    if col in df.columns:
        df[col] = df[col].astype("category")

print("✅ 型変換完了")



--- 型変換と日付整形 ---
🗑️ 'date_name' は 'date_' と重複していたため削除しました。
✅ 型変換完了


/tmp/ipykernel_16405/2267928510.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["date_"] = pd.to_datetime(df["date_"], errors="coerce")
/tmp/ipykernel_16405/2267928510.py:9: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  same_dates = (pd.to_datetime(df["date_name"], errors="coerce") == df["date_"]).all()


In [9]:
# start_time, end_time: UNIX秒→datetime変換
if np.issubdtype(df["start_time"].dtype, np.number):
    df["start_dt"] = pd.to_datetime(df["start_time"], unit="s", errors="coerce")
    df["end_dt"] = pd.to_datetime(df["end_time"], unit="s", errors="coerce")
    print("🕒 start_time / end_time を datetime に変換しました。")

🕒 start_time / end_time を datetime に変換しました。


In [10]:
# 時刻文字列列 (_h) が同じ情報なら削除
for col in ["start_time_h", "end_time_h"]:
    if col in df.columns:
        df = df.drop(columns=[col])
        print(f"🗑️ {col} を削除しました。")

🗑️ start_time_h を削除しました。
🗑️ end_time_h を削除しました。


In [11]:
# %% 列名統一
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)
print("✅ 列名をスネークケースに統一")


✅ 列名をスネークケースに統一


In [12]:
# %% 列の並び順整理
meta_cols = [
    "date_", "start_dt", "end_dt", "activity_id", "activity_name",
    "period_id", "period_name", "day_code",
    "athlete_id", "athlete_name", "is_injected"
]
other_cols = [c for c in df.columns if c not in meta_cols]
df = df[meta_cols + other_cols]

In [13]:
# セッション時間を追加（秒単位）
df["session_duration_sec"] = (df["end_dt"] - df["start_dt"]).dt.total_seconds()

In [14]:
# %% 列の順序整理
meta_cols = [
    "date_", "start_dt", "end_dt", "session_duration_sec",   # 時系列情報
    "activity_id", "activity_name", "period_id", "period_name", "day_code",  # セッション情報
    "athlete_id", "athlete_name", "is_injected"               # 選手情報
]

# 残りの測定値を追加
other_cols = [c for c in df.columns if c not in meta_cols]
df = df[meta_cols + other_cols]

print("✅ 列順序を整理しました（session_duration_sec を時刻列直後に配置）")

✅ 列順序を整理しました（session_duration_sec を時刻列直後に配置）


In [15]:
# %% 不可能値チェック：total_distance のみ削除
print("\n--- 不可能値（total_distance <= 0）の削除 ---")
invalid_distance = df["total_distance"] <= 0
removed = invalid_distance.sum()
df = df[~invalid_distance]
print(f"⚠️ total_distance <= 0 の行を削除しました: {removed} 行")



--- 不可能値（total_distance <= 0）の削除 ---
⚠️ total_distance <= 0 の行を削除しました: 177 行


In [16]:
# =====================================================
# GK判定フラグの作成（行動ベース・確定版）
# =====================================================

GK_COLS = [
    "dive_right_count",
    "dive_left_count",
    "dive_centre_count",
    "total_dive_load_centre",
    "total_dive_load_left",
    "total_dive_load_right",
    "total_time_to_feet_centre",
    "total_time_to_feet_left",
    "total_time_to_feet_right",
]

missing_gk_cols = [c for c in GK_COLS if c not in df.columns]
if missing_gk_cols:
    raise ValueError(f"❌ GK列が存在しません: {missing_gk_cols}")

# 行単位：GKイベントが1つでもあれば候補
df["is_gk_candidate"] = (df[GK_COLS].sum(axis=1) > 0).astype(int)

# 選手単位：GKイベント出現率（高速版）
gk_ratio = (
    df.groupby("athlete_id")["is_gk_candidate"]
      .mean()
)

# 「主にGK」とみなす閾値
GK_MAIN_TH = 0.8
gk_main_ids = gk_ratio[gk_ratio >= GK_MAIN_TH].index

# 最終GKフラグ
df["is_gk"] = df["athlete_id"].isin(gk_main_ids).astype(int)

# 分析用ポジション列
df["position_group"] = df["is_gk"].map({1: "GK", 0: "FP"}).astype("category")

print("✅ GK判定完了")
print(df["position_group"].value_counts())


✅ GK判定完了
position_group
FP    65471
GK     2523
Name: count, dtype: int64


In [17]:
# %% データ確認
print("\n--- データクリーニング後情報 ---")
print(f"{df.shape[0]} 行, {df.shape[1]} 列")
print(df.info())
print(df.head(3))



--- データクリーニング後情報 ---
67994 行, 74 列
<class 'pandas.core.frame.DataFrame'>
Index: 67994 entries, 0 to 68171
Data columns (total 74 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   date_                                           67994 non-null  datetime64[ns]
 1   start_dt                                        67994 non-null  datetime64[ns]
 2   end_dt                                          67994 non-null  datetime64[ns]
 3   session_duration_sec                            67994 non-null  float64       
 4   activity_id                                     67994 non-null  object        
 5   activity_name                                   67994 non-null  category      
 6   period_id                                       67994 non-null  object        
 7   period_name                                     67994 non-null  category      
 8   day_code       

In [18]:
# %%
# 保存先ディレクトリ（存在しなければ作成）
output_dir = "/workspace/data/00"
os.makedirs(output_dir, exist_ok=True)

# 出力ファイル名
csv_path = f"{output_dir}/StatsAllGroup_cleaned.csv"
parquet_path = f"{output_dir}/StatsAllGroup_cleaned.parquet"

# CSV保存
df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"💾 CSV保存完了: {csv_path}")

# Parquet保存（正式な downstream 用）
df.to_parquet(parquet_path, index=False)
print(f"⚡ Parquet保存完了: {parquet_path}")

# Downstream Notebook で参照する統一パス
CLEAN_PATH = parquet_path
print(f"✅ CLEAN_PATH = {CLEAN_PATH}")



💾 CSV保存完了: /workspace/data/00/StatsAllGroup_cleaned.csv
⚡ Parquet保存完了: /workspace/data/00/StatsAllGroup_cleaned.parquet
✅ CLEAN_PATH = /workspace/data/00/StatsAllGroup_cleaned.parquet
